In [64]:
from google.colab import drive
drive.mount('/content/drive/')

import os
os.chdir("/content/drive/MyDrive/ResearchProject")

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [65]:
from irp_functions import SMOTE
from irp_functions import Outlier_SMOTE

import pandas as pd
import numpy as np

In [66]:
root_path = '/content/drive/My Drive/ResearchProject/' 
dataset_path = root_path + 'Datasets/'
COVID_filename = root_path + 'Datasets/covid_dataset.csv'
df = pd.read_csv(COVID_filename)
df.head()

,Patient ID,Patient age quantile,SARS-Cov-2 exam result,"Patient addmited to regular ward (1=yes, 0=no)","Patient addmited to semi-intensive unit (1=yes, 0=no)","Patient addmited to intensive care unit (1=yes, 0=no)",Hematocrit,Hemoglobin,Platelets,Mean platelet volume,...,Hb saturation (arterial blood gases),pCO2 (arterial blood gas analysis),Base excess (arterial blood gas analysis),pH (arterial blood gas analysis),Total CO2 (arterial blood gas analysis),HCO3 (arterial blood gas analysis),pO2 (arterial blood gas analysis),Arteiral Fio2,Phosphor,ctO2 (arterial blood gas analysis)
0,44477f75e8169d2,13,negative,0,0,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,126e9dd13932f68,17,negative,0,0,0,0.236515,-0.02234,-0.517413,0.010677,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,a46b4402a0e5696,8,negative,0,0,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,f7d619a94f97c45,5,negative,0,0,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,d9e41465789c2b5,15,negative,0,0,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


SARS-Cov-2 exam result attribute

In [67]:
df['SARS-Cov-2 exam result'] = df['SARS-Cov-2 exam result'].map({'positive': 1,
                                                                 'negative': 0})

'Detected' & 'Not Detected' and 'Negative' & 'Positive' are to be mapped to binbary.

In [68]:
df = df.replace({'positive': 1, 'negative': 0,
                 'detected': 1, 'not_detected': 0})

Confirm imbalance ratio

In [69]:
df['SARS-Cov-2 exam result'].value_counts()

0    5086
1     558
Name: SARS-Cov-2 exam result, dtype: int64

Remove attributes where > 90% is null

In [70]:
df_null_percentage = df.isna().mean()

nulls = df_null_percentage[df_null_percentage > 0.9]

df = df[[col for col in df.columns if col not in nulls]]

df.head()

,Patient ID,Patient age quantile,SARS-Cov-2 exam result,"Patient addmited to regular ward (1=yes, 0=no)","Patient addmited to semi-intensive unit (1=yes, 0=no)","Patient addmited to intensive care unit (1=yes, 0=no)",Hematocrit,Hemoglobin,Platelets,Mean platelet volume,...,Adenovirus,Parainfluenza 4,Coronavirus229E,CoronavirusOC43,Inf A H1N1 2009,Bordetella pertussis,Metapneumovirus,Parainfluenza 2,"Influenza B, rapid test","Influenza A, rapid test"
0,44477f75e8169d2,13,0,0,0,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,126e9dd13932f68,17,0,0,0,0,0.236515,-0.02234,-0.517413,0.010677,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,a46b4402a0e5696,8,0,0,0,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,f7d619a94f97c45,5,0,0,0,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,d9e41465789c2b5,15,0,0,0,0,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN


Find variances

In [71]:
df.var()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  """Entry point for launching an IPython kernel.


Patient age quantile                                     33.384166
SARS-Cov-2 exam result                                    0.089107
Patient addmited to regular ward (1=yes, 0=no)            0.013804
Patient addmited to semi-intensive unit (1=yes, 0=no)     0.008782
Patient addmited to intensive care unit (1=yes, 0=no)     0.007213
Hematocrit                                                1.001661
Hemoglobin                                                1.001661
Platelets                                                 1.001664
Mean platelet volume                                      1.001672
Red blood Cells                                           1.001664
Lymphocytes                                               1.001664
Mean corpuscular hemoglobin concentration (MCHC)          1.001664
Leukocytes                                                1.001664
Basophils                                                 1.001664
Mean corpuscular hemoglobin (MCH)                         1.00

Remove attributes withg zero variance

In [72]:
df = df.loc[:,df.apply(pd.Series.nunique) != 1]
df.head()

,Patient ID,Patient age quantile,SARS-Cov-2 exam result,"Patient addmited to regular ward (1=yes, 0=no)","Patient addmited to semi-intensive unit (1=yes, 0=no)","Patient addmited to intensive care unit (1=yes, 0=no)",Hematocrit,Hemoglobin,Platelets,Mean platelet volume,...,Chlamydophila pneumoniae,Adenovirus,Parainfluenza 4,Coronavirus229E,CoronavirusOC43,Inf A H1N1 2009,Bordetella pertussis,Metapneumovirus,"Influenza B, rapid test","Influenza A, rapid test"
0,44477f75e8169d2,13,0,0,0,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,126e9dd13932f68,17,0,0,0,0,0.236515,-0.02234,-0.517413,0.010677,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,a46b4402a0e5696,8,0,0,0,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,f7d619a94f97c45,5,0,0,0,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,d9e41465789c2b5,15,0,0,0,0,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN


Create a "has disease" variable from the last 19 columns

In [73]:
df['has_disease'] = df[df.columns[20:]].sum(axis=1)

df.loc[df['has_disease'] > 1, 'has_disease'] = 1

df['has_disease'].value_counts(normalize=True)

0.0    0.865344
1.0    0.134656
Name: has_disease, dtype: float64

Imputation of nan disease variables to 0.
Assume if no affirmative answer to presence of a disease (presumably not tested) then don't have the disease.

In [74]:
df[df.columns[20:]] = df[df.columns[20:]].fillna(0)

Imputation of other nan variables to mean as per Turlapti & Prusty

In [75]:
df = df.fillna(df.mean())
df.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  """Entry point for launching an IPython kernel.


,Patient ID,Patient age quantile,SARS-Cov-2 exam result,"Patient addmited to regular ward (1=yes, 0=no)","Patient addmited to semi-intensive unit (1=yes, 0=no)","Patient addmited to intensive care unit (1=yes, 0=no)",Hematocrit,Hemoglobin,Platelets,Mean platelet volume,...,Adenovirus,Parainfluenza 4,Coronavirus229E,CoronavirusOC43,Inf A H1N1 2009,Bordetella pertussis,Metapneumovirus,"Influenza B, rapid test","Influenza A, rapid test",has_disease
0,44477f75e8169d2,13,0,0,0,0,-2.186214e-09,-1.601319e-08,-3.535004e-10,7.438143e-09,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,126e9dd13932f68,17,0,0,0,0,2.365154e-01,-2.234027e-02,-5.174130e-01,1.067657e-02,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,a46b4402a0e5696,8,0,0,0,0,-2.186214e-09,-1.601319e-08,-3.535004e-10,7.438143e-09,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,f7d619a94f97c45,5,0,0,0,0,-2.186214e-09,-1.601319e-08,-3.535004e-10,7.438143e-09,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,d9e41465789c2b5,15,0,0,0,0,-2.186214e-09,-1.601319e-08,-3.535004e-10,7.438143e-09,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [76]:
X = df.drop(['SARS-Cov-2 exam result', 'Patient ID'], axis=1)
X_columns = X.columns
y = df['SARS-Cov-2 exam result']

In [77]:
X.dtypes

Patient age quantile                                       int64
Patient addmited to regular ward (1=yes, 0=no)             int64
Patient addmited to semi-intensive unit (1=yes, 0=no)      int64
Patient addmited to intensive care unit (1=yes, 0=no)      int64
Hematocrit                                               float64
Hemoglobin                                               float64
Platelets                                                float64
Mean platelet volume                                     float64
Red blood Cells                                          float64
Lymphocytes                                              float64
Mean corpuscular hemoglobin concentration (MCHC)         float64
Leukocytes                                               float64
Basophils                                                float64
Mean corpuscular hemoglobin (MCH)                        float64
Eosinophils                                              float64
Mean corpuscular volume (

In [78]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_unscaled = X

scaler.fit(X)
X = scaler.transform(X)

In [87]:
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import confusion_matrix

skf = StratifiedKFold(n_splits=3, shuffle=True, random_state=1)
for train_index, test_index in skf.split(X, y):
  X_train, X_test = X[train_index], X[test_index]
  y_train, y_test = y[train_index], y[test_index]

  pos_indices = np.where(y_train == 1)
  X_train_positive = np.take(X_train, pos_indices, axis=0)[0]
  mySMOTE = SMOTE(X_train_positive, 2000, 5)
  X_synthetic = mySMOTE.oversample()
  n_synthetic = np.shape(X_synthetic)[0]
  y_synthetic = np.ones(n_synthetic)

  X_train = np.concatenate((X_train, X_synthetic))
  y_train = np.concatenate((y_train, y_synthetic))

  regr = RandomForestRegressor(max_depth=50, random_state=0, n_estimators=100)
  regr.fit(X_train, y_train)

  y_predicted_probabilities = regr.predict(X_test)
  y_predicted_classes = np.where(y_predicted_probabilities > 0.5, 1, 0)

  print(y_pred)

  print(confusion_matrix(y_test, y_predicted_classes))





 
  

[0.30555309 0.30464229 0.01019284 ... 0.18681717 0.30442156 0.91828916]
[[ 588 1108]
 [  21  165]]
[0.30555309 0.30464229 0.01019284 ... 0.18681717 0.30442156 0.91828916]
[[ 522 1173]
 [  12  174]]
[0.30555309 0.30464229 0.01019284 ... 0.18681717 0.30442156 0.91828916]
[[ 536 1159]
 [  15  171]]
